In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from matplotlib.colors import(PowerNorm,LogNorm)

In [ ]:
R = 'Ha'
G = 'V'
B = 'B'

In [ ]:
r = fits.getdata('./Output/M51_aligned/%s_final_norm_align.fits' % R)
g = fits.getdata('./Output/M51_aligned/%s_final_norm_align.fits' % G)
b = fits.getdata('./Output/M51_aligned/%s_final_norm_align.fits' % B)

In [ ]:
low_1 = 0.5 
high_1 = 4

low_2 = 0.1 
high_2 = 4

rmin_1, rmax_1 = np.mean(r)+low_1*np.std(r), np.mean(r)+high_1*np.std(r)
gmin_1, gmax_1 = np.mean(g)+low_1*np.std(g), np.mean(g)+high_1*np.std(g)
bmin_1, bmax_1 = np.mean(b)+low_1*np.std(b), np.mean(b)+high_1*np.std(b)

rmin_2, rmax_2 = np.mean(r)+low_2*np.std(r), np.mean(r)+high_2*np.std(r)
gmin_2, gmax_2 = np.mean(g)+low_2*np.std(g), np.mean(g)+high_2*np.std(g)
bmin_2, bmax_2 = np.mean(b)+low_2*np.std(b), np.mean(b)+high_2*np.std(b)

img_1 = np.zeros((1024,1024,3))
img_2 = np.zeros((1024,1024,3))

img_1[:,:,0]=asinh(r,scale_min=rmin_1,scale_max=rmax_1)
img_1[:,:,1]=asinh(g,scale_min=gmin_1,scale_max=gmax_1)
img_1[:,:,2]=asinh(b,scale_min=bmin_1,scale_max=bmax_1)

img_2[:,:,0]=asinh(r,scale_min=rmin_2,scale_max=rmax_2)
img_2[:,:,1]=asinh(g,scale_min=gmin_2,scale_max=gmax_2)
img_2[:,:,2]=asinh(b,scale_min=bmin_2,scale_max=bmax_2)

In [ ]:
fig=plt.figure(figsize=(20,20))
plt.subplot(121)
plt.axis('off')
plt.imshow(img_1, origin='lower')
plt.subplot(122)
plt.imshow(img_2, origin='lower')
plt.axis('off')
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,15))
plt.axis('off')
plt.imshow(img_1, origin='lower')
plt.savefig('./Output/Colors/%s%s%s_norm.pdf' % (R,G,B),bbox_inches='tight')
plt.savefig('./Output/Colors/%s%s%s_norm.jpg' % (R,G,B),bbox_inches='tight')
plt.show()

In [ ]:
#Doesn't really do anything
hdu = fits.PrimaryHDU(img)
hdul = fits.HDUList([hdu])
#hdu.writeto('./Output/Color_%s%s%s.fits' % (R,G,B), overwrite='True')

In [ ]:
def asinh(inputArray, scale_min=None, scale_max=None, non_linear=2.0):
	"""Performs asinh scaling of the input numpy array.

	@type inputArray: numpy array
	@param inputArray: image data array
	@type scale_min: float
	@param scale_min: minimum data value
	@type scale_max: float
	@param scale_max: maximum data value
	@type non_linear: float
	@param non_linear: non-linearity factor
	@rtype: numpy array
	@return: image data array
	
	"""		
    
	#print "img_scale : asinh"
	imageData=np.array(inputArray, copy=True)
	
	if scale_min == None:
		scale_min = imageData.min()
	if scale_max == None:
		scale_max = imageData.max()
	factor = np.arcsinh((scale_max - scale_min)/non_linear)
	indices0 = np.where(imageData < scale_min)
	indices1 = np.where((imageData >= scale_min) & (imageData <= scale_max))
	indices2 = np.where(imageData > scale_max)
	imageData[indices0] = 0.0
	imageData[indices2] = 1.0
	imageData[indices1] = np.arcsinh((imageData[indices1] - \
	scale_min)/non_linear)/factor

	return imageData